In [ ]:
!pip install bayesian-optimization
!pip install keras-tuner
!pip install catboost
!pip install vecstack

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rc('font',family='malgun gothic')
plt.rc('axes',unicode_minus=False)
import seaborn as sns

# Preprocessing&Feature Engineering
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, RobustScaler, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile
from gensim.models import word2vec

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
import kerastuner as kt

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, ARDRegression, BayesianRidge, Lars
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from vecstack import StackingTransformer
from vecstack import stacking

# Eveluation
from sklearn.metrics import mean_squared_error # squared=False시 RMSE
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import pickle

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/D&A_ML_Competition/L.POINT_train.csv', encoding='UTF-8')
test = pd.read_csv('/content/drive/MyDrive/D&A_ML_Competition/L.POINT_test.csv', encoding='UTF-8')
y_target = pd.read_csv('/content/drive/MyDrive/D&A_ML_Competition/y_train.csv')

In [ ]:
train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
train['TOT_SESS_HR_V']  = train['TOT_SESS_HR_V'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['TOT_SESS_HR_V']  = test['TOT_SESS_HR_V'].map(lambda x: int(str(x).replace(',','')) )

In [ ]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [ ]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [ ]:
features = []
features_te = []

#### 지역 대분류

In [ ]:
# 지역 대분류별 방문횟수
f = pd.crosstab(train.CLNT_ID,train.ZON_NM)
f = f.rename(columns=dict(zip(f.columns,[x+'_횟수' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.crosstab(test.CLNT_ID,test.ZON_NM)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_횟수' for x in f_te.columns]))).reset_index()
features_te.append(f_te) ; display(f_te)

In [ ]:
# 최대 방문 지역 대분류
f = train.groupby('CLNT_ID').ZON_NM.max().reset_index()
f = pd.crosstab(f.CLNT_ID,f.ZON_NM)
f = f.rename(columns=dict(zip(f.columns,['최대지역_'+x for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').ZON_NM.max().reset_index()
f_te = pd.crosstab(f_te.CLNT_ID,f_te.ZON_NM)
f_te = f_te.rename(columns=dict(zip(f_te.columns,['최대지역_'+x for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 대분류별 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='sum').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='sum').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 대분류별 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='mean').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='mean').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 대분류별 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='max').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='max').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 대분류별 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='min').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='min').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

#### 지역 중분류

In [ ]:
# 지역 중분류별 방문횟수
f = pd.crosstab(train.CLNT_ID,train.CITY_NM)
f = f.rename(columns=dict(zip(f.columns,[x+'_횟수' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.crosstab(test.CLNT_ID,test.CITY_NM)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_횟수' for x in f_te.columns]))).reset_index()
features_te.append(f_te) ; display(f_te)

In [ ]:
# 최대 방문 지역 중분류
f = train.groupby('CLNT_ID').CITY_NM.max().reset_index()
f = pd.crosstab(f.CLNT_ID,f.CITY_NM)
f = f.rename(columns=dict(zip(f.columns,['최대지역_'+x for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').CITY_NM.max().reset_index()
f_te = pd.crosstab(f_te.CLNT_ID,f_te.CITY_NM)
f_te = f_te.rename(columns=dict(zip(f_te.columns,['최대지역_'+x for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 중분류별 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='sum').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='sum').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 중분류별 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='mean').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='mean').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 중분류별 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='max').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='max').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 지역 중분류별 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='min').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='min').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

#### 세션 아이디

In [ ]:
# 구매 세션 개수
f = train.groupby('CLNT_ID').SESS_ID.agg([('구매세션_개수','nunique')]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').SESS_ID.agg([('구매세션_개수','nunique')]).reset_index()
features_te.append(f_te);display(f_te)

#### 히트일련번호

In [ ]:
# 평균_최초구매_히트횟수
f = train.groupby(['CLNT_ID','SESS_ID']).HITS_SEQ.min().groupby('CLNT_ID').mean().reset_index()\
            .rename(columns={'HITS_SEQ':'평균_최초구매_히트횟수'})
features.append(f);display(f)

f_te = test.groupby(['CLNT_ID','SESS_ID']).HITS_SEQ.min().groupby('CLNT_ID').mean().reset_index()\
            .rename(columns={'HITS_SEQ':'평균_최초구매_히트횟수'})
features_te.append(f_te);display(f_te)

In [ ]:
# 최대_최초구매_히트횟수
f = train.groupby(['CLNT_ID','SESS_ID']).HITS_SEQ.min().groupby('CLNT_ID').max().reset_index()\
            .rename(columns={'HITS_SEQ':'최대_최초구매_히트횟수'})
features.append(f);display(f)

f_te = test.groupby(['CLNT_ID','SESS_ID']).HITS_SEQ.min().groupby('CLNT_ID').max().reset_index()\
            .rename(columns={'HITS_SEQ':'최대_최초구매_히트횟수'})
features_te.append(f_te);display(f_te)

In [ ]:
# 최소_최초구매_히트횟수
f = train.groupby(['CLNT_ID','SESS_ID']).HITS_SEQ.min().groupby('CLNT_ID').min().reset_index()\
            .rename(columns={'HITS_SEQ':'최소_최초구매_히트횟수'})
features.append(f);display(f)

f_te = test.groupby(['CLNT_ID','SESS_ID']).HITS_SEQ.min().groupby('CLNT_ID').min().reset_index()\
            .rename(columns={'HITS_SEQ':'최소_최초구매_히트횟수'})
features_te.append(f_te);display(f_te)

#### 세션일자

In [ ]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [ ]:
# 세션 일자 간격
def diff_sess_dt(x):
    return x.diff().fillna(0).mean()
f = train[['CLNT_ID','SESS_DT']].sort_values(['CLNT_ID','SESS_DT']).groupby('CLNT_ID').SESS_DT.agg(
                [('세션_일자_간격',diff_sess_dt)]).reset_index()
features.append(f); display(f)

f_te = test[['CLNT_ID','SESS_DT']].sort_values(['CLNT_ID','SESS_DT']).groupby('CLNT_ID').SESS_DT.agg(
                [('세션_일자_간격',diff_sess_dt)]).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 일 별 평균 세션 개수
f = train.groupby(['CLNT_ID','date']).SESS_ID.nunique().reset_index().groupby('CLNT_ID').SESS_ID.agg([('일별_평균_세션수','mean')]).reset_index()
features.append(f); display(f)

f_te = test.groupby(['CLNT_ID','date']).SESS_ID.nunique().reset_index().groupby('CLNT_ID').SESS_ID.agg([('일별_평균_세션수','mean')]).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
f = train.groupby('CLNT_ID')['SESS_DT'].agg([('내점일수', lambda x: x.nunique())])
f = f.reset_index()
f = f.fillna(0)
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['SESS_DT'].agg([('내점일수', lambda x: x.nunique())])
f_te = f_te.reset_index()
f_te = f_te.fillna(0)
features_te.append(f_te);display(f_te)

#### 요일별

In [ ]:
# 요일 별 구매 횟수
f = pd.crosstab(train.CLNT_ID,train.date.dt.day_name())
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_방문횟수' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.crosstab(test.CLNT_ID,test.date.dt.day_name())
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_방문횟수' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 요일 별 총 구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.day_name(), values='AMOUNT', aggfunc='sum', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.day_name(), values='AMOUNT', aggfunc='sum', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 요일 별 평균 구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.day_name(), values='AMOUNT', aggfunc='mean', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.day_name(), values='AMOUNT', aggfunc='mean', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 요일 별 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.day_name(), values='AMOUNT', aggfunc='max', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.day_name(), values='AMOUNT', aggfunc='max', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 요일 별 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.day_name(), values='AMOUNT', aggfunc='min', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.day_name(), values='AMOUNT', aggfunc='min', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

#### 평일/주말 별

In [ ]:
# 평일 방문 횟수
f = train.groupby('CLNT_ID')['date'].agg([
    ('평일방문횟수', lambda x: np.size(x.dt.dayofweek<=4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('평일방문횟수', lambda x: np.size(x.dt.dayofweek<=4))]).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 주말 방문 횟수
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문횟수', lambda x: np.size(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문횟수', lambda x: np.size(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 평일/주말 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='sum', fill_value=0)
f = f.rename(columns={False:'평일_총구매금액',True:'주말_총구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='sum', fill_value=0)
f_te = f_te.rename(columns={False:'평일_총구매금액',True:'주말_총구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 평일/주말 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='mean', fill_value=0)
f = f.rename(columns={False:'평일_평균구매금액',True:'주말_평균구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='mean', fill_value=0)
f_te = f_te.rename(columns={False:'평일_평균구매금액',True:'주말_평균구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 평일/주말 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='max', fill_value=0)
f = f.rename(columns={False:'평일_최대구매금액',True:'주말_최대구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='max', fill_value=0)
f_te = f_te.rename(columns={False:'평일_최대구매금액',True:'주말_최대구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 평일/주말 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='min', fill_value=0)
f = f.rename(columns={False:'평일_최소구매금액',True:'주말_최소구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.dayofweek>4, values='AMOUNT', aggfunc='min', fill_value=0)
f_te = f_te.rename(columns={False:'평일_최소구매금액',True:'주말_최소구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

#### 월 초/말 별

In [ ]:
# 월 초 방문 횟수
f = train.groupby('CLNT_ID')['date'].agg([
    ('월초_방문횟수', lambda x: np.size(x.dt.day<=15))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('월초_방문횟수', lambda x: np.size(x.dt.day<=15))]).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월 말 방문 횟수
f = train.groupby('CLNT_ID')['date'].agg([
    ('월말_방문횟수', lambda x: np.size(x.dt.day>15))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('월말_방문횟수', lambda x: np.size(x.dt.day>15))]).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월초/월말 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.day>15, values='AMOUNT', aggfunc='sum', fill_value=0)
f = f.rename(columns={False:'월초_총구매금액',True:'월말_총구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.day>15, values='AMOUNT', aggfunc='sum', fill_value=0)
f_te = f_te.rename(columns={False:'월초_총구매금액',True:'월말_총구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월초/월말 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.day>15, values='AMOUNT', aggfunc='mean', fill_value=0)
f = f.rename(columns={False:'월초_평균구매금액',True:'월말_평균구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.day>15, values='AMOUNT', aggfunc='mean', fill_value=0)
f_te = f_te.rename(columns={False:'월초_평균구매금액',True:'월말_평균구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월초/월말 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.day>15, values='AMOUNT', aggfunc='max', fill_value=0)
f = f.rename(columns={False:'월초_최대구매금액',True:'월말_최대구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.day>15, values='AMOUNT', aggfunc='max', fill_value=0)
f_te = f_te.rename(columns={False:'월초_최대구매금액',True:'월말_최대구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월초/월말 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.day>15, values='AMOUNT', aggfunc='min', fill_value=0)
f = f.rename(columns={False:'월초_최소구매금액',True:'월말_최소구매금액'}).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.day>15, values='AMOUNT', aggfunc='min', fill_value=0)
f_te = f_te.rename(columns={False:'월초_최소구매금액',True:'월말_최소구매금액'}).reset_index()
features_te.append(f_te); display(f_te)

#### 월별

In [ ]:
# 월 별 구매 횟수
f = pd.crosstab(train.CLNT_ID,train.date.dt.month)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'월_방문횟수' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.crosstab(test.CLNT_ID,test.date.dt.month)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'월_방문횟수' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월 별 총 구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.month, values='AMOUNT', aggfunc='sum', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.month, values='AMOUNT', aggfunc='sum', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월 별 평균 구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.month, values='AMOUNT', aggfunc='mean', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.month, values='AMOUNT', aggfunc='mean', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월 별 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.month, values='AMOUNT', aggfunc='max', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.month, values='AMOUNT', aggfunc='max', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 월 별 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.month, values='AMOUNT', aggfunc='min', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.month, values='AMOUNT', aggfunc='min', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

#### 계절별

In [ ]:
# 계절별 구매 횟수
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매횟수', lambda x: np.size( x.dt.month.isin([3,4,5]))),
    ('여름-구매횟수', lambda x: np.size( x.dt.month.isin([6,7,8]))),
    ('가을-구매횟수', lambda x: np.size(x.dt.month.isin([9,10,11]))),
    ('겨울-구매횟수', lambda x: np.size( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매횟수', lambda x: np.size( x.dt.month.isin([3,4,5]))),
    ('여름-구매횟수', lambda x: np.size( x.dt.month.isin([6,7,8]))),
    ('가을-구매횟수', lambda x: np.size(x.dt.month.isin([9,10,11]))),
    ('겨울-구매횟수', lambda x: np.size( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

In [ ]:
# 계절 별 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([3,4,5]),'봄',
        np.where(train.date.dt.month.isin([6,7,8]),'여름',
        np.where(train.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='sum', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([3,4,5]),'봄',
        np.where(test.date.dt.month.isin([6,7,8]),'여름',
        np.where(test.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='sum', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 계절 별 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([3,4,5]),'봄',
        np.where(train.date.dt.month.isin([6,7,8]),'여름',
        np.where(train.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='mean', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([3,4,5]),'봄',
        np.where(test.date.dt.month.isin([6,7,8]),'여름',
        np.where(test.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='mean', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 계절 별 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([3,4,5]),'봄',
        np.where(train.date.dt.month.isin([6,7,8]),'여름',
        np.where(train.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='max', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([3,4,5]),'봄',
        np.where(test.date.dt.month.isin([6,7,8]),'여름',
        np.where(test.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='max', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 계절 별 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([3,4,5]),'봄',
        np.where(train.date.dt.month.isin([6,7,8]),'여름',
        np.where(train.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='min', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([3,4,5]),'봄',
        np.where(test.date.dt.month.isin([6,7,8]),'여름',
        np.where(test.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='min', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

#### 반기별

In [ ]:
# 반기 별 구매 횟수
f = train.groupby('CLNT_ID')['date'].agg([
    ('전반기-구매횟수', lambda x: np.size( x.dt.month.isin([1,2,3,4,5,6]))),
    ('후반기-구매횟수', lambda x: np.size( x.dt.month.isin([7,8,9,10,11,12])))
]).reset_index()
features.append(f); f

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('전반기-구매횟수', lambda x: np.size( x.dt.month.isin([1,2,3,4,5,6]))),
    ('후반기-구매횟수', lambda x: np.size( x.dt.month.isin([7,8,9,10,11,12])))
]).reset_index()
features_te.append(f_te); f_te

In [ ]:
# 전반기/후반기 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                  '후반기'), values='AMOUNT', aggfunc='sum', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                    '후반기'), values='AMOUNT', aggfunc='sum', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 전반기/후반기 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                  '후반기'), values='AMOUNT', aggfunc='mean', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                    '후반기'), values='AMOUNT', aggfunc='mean', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 전반기/후반기 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                  '후반기'), values='AMOUNT', aggfunc='max', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                    '후반기'), values='AMOUNT', aggfunc='max', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

In [ ]:
# 전반기/후반기 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                  '후반기'), values='AMOUNT', aggfunc='min', fill_value=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                    '후반기'), values='AMOUNT', aggfunc='min', fill_value=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te); display(f_te)

#### 총 페이지 조회 건수

In [ ]:
# 총 페이지 조회 건수 평균, 최대, 최소
f = train.groupby('CLNT_ID').TOT_PAG_VIEW_CT.agg(
    [('총페이지조회건수_평균','mean'),('총페이지조회건수_최대','max'),('총페이지조회건수_최소','min')]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').TOT_PAG_VIEW_CT.agg(
    [('총페이지조회건수_평균','mean'),('총페이지조회건수_최대','max'),('총페이지조회건수_최소','min')]).reset_index()
features_te.append(f_te);display(f_te)

#### 총 세션 시간

In [ ]:
# 총 세션 시간 평균, 최대, 최소
f = train.groupby('CLNT_ID').TOT_SESS_HR_V.agg(
    [('총세션시간_평균','mean'),('총세션시간_최대','max'),('총세션시간_최소','min')]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').TOT_SESS_HR_V.agg(
    [('총세션시간_평균','mean'),('총세션시간_최대','max'),('총세션시간_최소','min')]).reset_index()
features_te.append(f_te);display(f_te)

#### 기기유형

In [ ]:
# 기기유형별 방문 횟수
f = pd.crosstab(train.CLNT_ID, train.DVC_CTG_NM)
f = f.rename(columns=dict(zip(f.columns,[x+'_구매횟수' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.crosstab(test.CLNT_ID, test.DVC_CTG_NM)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_구매횟수' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 사용 기기 개수
f = train.groupby('CLNT_ID').DVC_CTG_NM.agg([('사용기기개수','nunique')]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').DVC_CTG_NM.agg([('사용기기개수','nunique')]).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 사용기기별 총구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='sum').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_총구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='sum').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_총구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 사용기기별 평균구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='mean').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_평균구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='mean').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_평균구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 사용기기별 최대구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='max').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_최대구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='max').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_최대구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

In [ ]:
# 사용기기별 최소구매금액
f = pd.pivot_table(data = train, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='min').fillna(0)
f = f.rename(columns=dict(zip(f.columns,[x+'_최소구매금액' for x in f.columns]))).reset_index()
features.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='min').fillna(0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_최소구매금액' for x in f_te.columns]))).reset_index()
features_te.append(f_te);display(f_te)

#### 상품 구매건수

In [ ]:
f = train.groupby('CLNT_ID').PD_BUY_CT.agg([('상품_평균_구매개수','mean'),('상품_최대_구매개수','max')]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').PD_BUY_CT.agg([('상품_평균_구매개수','mean'),('상품_최대_구매개수','max')]).reset_index()
features_te.append(f_te);display(f_te)

#### 상품 대, 중, 소분류

In [ ]:
cust_train = train.CLNT_ID.unique()
cust_test = test.CLNT_ID.unique()
concat_data = pd.concat([train,test])

f1_list = train.CLAC1_NM.unique()
f2_list = train.CLAC2_NM.unique()
f3_list = train.CLAC3_NM.unique()

In [ ]:
f1 = pd.crosstab(concat_data.CLNT_ID, concat_data.CLAC1_NM)[f1_list]
f1 = f1.rename(columns=dict(zip(f1.columns,[x+'_구매개수' for x in f1.columns]))).reset_index()
f2 = pd.crosstab(concat_data.CLNT_ID, concat_data.CLAC2_NM)[f2_list]
f2 = f2.rename(columns=dict(zip(f2.columns,[x+'_구매개수' for x in f2.columns]))).reset_index()
#f3 = pd.crosstab(concat_data.CLNT_ID, concat_data.CLAC3_NM)[f3_list]
#f3 = f3.rename(columns=dict(zip(f3.columns,[x+'_구매개수' for x in f3.columns]))).reset_index()

In [ ]:
# 상품 대분류별 구매 횟수
f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')

features.append(f);display(f)
features_te.append(f_te);display(f_te)

In [ ]:
# 상품 대분류별 구매 여부
def yes_or_no(x):
    if x != 0:
        return 1
    else:
        return 0
f.iloc[:,1:] = f.iloc[:,1:].applymap(yes_or_no)
f = f.rename(columns=dict(zip(f.columns,[x.replace('개수','여부') for x in f.columns])))
features.append(f);display(f)

f_te.iloc[:,1:] = f_te.iloc[:,1:].applymap(yes_or_no)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x.replace('개수','여부') for x in f_te.columns])))
features_te.append(f_te);display(f_te)

In [ ]:
# 상품 중분류별 구매 횟수
f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')

features.append(f);display(f)
features_te.append(f_te);display(f_te)

In [ ]:
# 상품 중분류별 구매 여부
f.iloc[:,1:] = f.iloc[:,1:].applymap(yes_or_no)
f = f.rename(columns=dict(zip(f.columns,[x.replace('개수','여부') for x in f.columns])))
features.append(f);display(f)

f_te.iloc[:,1:] = f_te.iloc[:,1:].applymap(yes_or_no)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x.replace('개수','여부') for x in f_te.columns])))
features_te.append(f_te);display(f_te)

In [ ]:
# 상품 소분류별 구매 횟수
#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')

#features.append(f);display(f)
#features_te.append(f_te);display(f_te)

In [ ]:
# 상품 소분류별 구매 여부
#f.iloc[:,1:] = f.iloc[:,1:].applymap(yes_or_no)
#f = f.rename(columns=dict(zip(f.columns,[x.replace('개수','여부') for x in f.columns])))
#features.append(f);display(f)

#f_te.iloc[:,1:] = f_te.iloc[:,1:].applymap(yes_or_no)
#f_te = f_te.rename(columns=dict(zip(f_te.columns,[x.replace('개수','여부') for x in f_te.columns])))
#features_te.append(f_te);display(f_te)

In [ ]:
# 상품 대,중,소분류별 총구매금액
f1 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC1_NM',values='AMOUNT', aggfunc='sum').fillna(0)[f1_list]
f1 = f1.rename(columns=dict(zip(f1.columns,[x+'_총구매금액' for x in f1.columns]))).reset_index()
f2 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC2_NM',values='AMOUNT', aggfunc='sum').fillna(0)[f2_list]
f2 = f2.rename(columns=dict(zip(f2.columns,[x+'_총구매금액' for x in f2.columns]))).reset_index()
#f3 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC3_NM',values='AMOUNT', aggfunc='sum').fillna(0)[f3_list]
#f3 = f3.rename(columns=dict(zip(f3.columns,[x+'_총구매금액' for x in f3.columns]))).reset_index()

f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')
#features.append(f);display(f)
#features_te.append(f_te);display(f_te)

In [ ]:
# 상품 대,중,소분류별 평균구매금액
f1 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC1_NM',values='AMOUNT', aggfunc='mean').fillna(0)[f1_list]
f1 = f1.rename(columns=dict(zip(f1.columns,[x+'_평균구매금액' for x in f1.columns]))).reset_index()
f2 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC2_NM',values='AMOUNT', aggfunc='mean').fillna(0)[f2_list]
f2 = f2.rename(columns=dict(zip(f2.columns,[x+'_평균구매금액' for x in f2.columns]))).reset_index()
#f3 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC3_NM',values='AMOUNT', aggfunc='mean').fillna(0)[f3_list]
#f3 = f3.rename(columns=dict(zip(f3.columns,[x+'_평균구매금액' for x in f3.columns]))).reset_index()

f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')
#features.append(f);display(f)
#features_te.append(f_te);display(f_te)

In [ ]:
# 상품 대,중,소분류별 최대구매금액
f1 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC1_NM',values='AMOUNT', aggfunc='max').fillna(0)[f1_list]
f1 = f1.rename(columns=dict(zip(f1.columns,[x+'_최대구매금액' for x in f1.columns]))).reset_index()
f2 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC2_NM',values='AMOUNT', aggfunc='max').fillna(0)[f2_list]
f2 = f2.rename(columns=dict(zip(f2.columns,[x+'_최대구매금액' for x in f2.columns]))).reset_index()
#f3 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC3_NM',values='AMOUNT', aggfunc='max').fillna(0)[f3_list]
#f3 = f3.rename(columns=dict(zip(f3.columns,[x+'_최대구매금액' for x in f3.columns]))).reset_index()

f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')
#features.append(f);display(f)
#features_te.append(f_te);display(f_te)

In [ ]:
# 상품 대,중,소분류별 최소구매금액
f1 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC1_NM',values='AMOUNT', aggfunc='min').fillna(0)[f1_list]
f1 = f1.rename(columns=dict(zip(f1.columns,[x+'_최소구매금액' for x in f1.columns]))).reset_index()
f2 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC2_NM',values='AMOUNT', aggfunc='min').fillna(0)[f2_list]
f2 = f2.rename(columns=dict(zip(f2.columns,[x+'_최소구매금액' for x in f2.columns]))).reset_index()
#f3 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC3_NM',values='AMOUNT', aggfunc='min').fillna(0)[f3_list]
#f3 = f3.rename(columns=dict(zip(f3.columns,[x+'_최소구매금액' for x in f3.columns]))).reset_index()

f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')
features.append(f);display(f)
features_te.append(f_te);display(f_te)

#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')
#features.append(f);display(f)
#features_te.append(f_te);display(f_te)

#### 검색 건 수

In [ ]:
# 검색 건 수 최대, 평균
f = train.groupby('CLNT_ID').SEARCH_CNT.agg([('검색건수_평균','mean'),('검색건수_최대','max')]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID').SEARCH_CNT.agg([('검색건수_평균','mean'),('검색건수_최대','max')]).reset_index()
features_te.append(f_te);display(f_te)

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [ ]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max),
                                            ('최소구매액', np.min)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max),
                                            ('최소구매액', np.min)]).reset_index()
features_te.append(f_te) ; display(f_te)

# <font color='CC3D3D'> Feature Merge

In [ ]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})

for f in tqdm(features) :
    data = pd.merge(data, f, how='left', on='CLNT_ID')

In [ ]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})

for f in tqdm(features_te) :
    data_te = pd.merge(data_te, f, how='left', on='CLNT_ID')

In [ ]:
data.to_csv('data1.csv')
data_te.to_csv('data_te1.csv')

# <font color='CC3D3D'> Feature Merge

### 비율 변수

In [ ]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [ ]:
features_prop = []
features_te_prop = []

In [ ]:
# 지역 대분류별 방문비율
f = pd.crosstab(train.CLNT_ID,train.ZON_NM,margins=True)
f = f.div(f.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f.columns,[x+'_비율' for x in f.columns]))).reset_index()
features_prop.append(f);display(f)

f_te = pd.crosstab(test.CLNT_ID,test.ZON_NM,margins=True)
f_te = f_te.div(f_te.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f_te.columns,[x+'_비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te) ; display(f_te)

In [ ]:
# 지역 중분류별 방문비율
f = pd.crosstab(train.CLNT_ID,train.CITY_NM,margins=True)
f = f.div(f.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f.columns,[x+'_비율' for x in f.columns]))).reset_index()
features_prop.append(f);display(f)

f_te = pd.crosstab(test.CLNT_ID,test.CITY_NM,margins=True)
f_te = f_te.div(f_te.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f_te.columns,[x+'_비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te) ; display(f_te)

In [ ]:
# 요일 별 구매 비율
f = pd.crosstab(train.CLNT_ID,train.date.dt.day_name(),margins=True)
f = f.div(f.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f.columns,[x+'_방문비율' for x in f.columns]))).reset_index()
features_prop.append(f); display(f)

f_te = pd.crosstab(test.CLNT_ID,test.date.dt.day_name(),margins=True)
f_te = f_te.div(f_te.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f_te.columns,[x+'_방문비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 평일 방문 비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('평일방문비율', lambda x: np.mean(x.dt.dayofweek<=4))]).reset_index()
features_prop.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('평일방문비율', lambda x: np.mean(x.dt.dayofweek<=4))]).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 주말 방문 비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_prop.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 월 초 방문 비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('월초_방문비율', lambda x: np.mean(x.dt.day<=15))]).reset_index()
features_prop.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('월초_방문비율', lambda x: np.mean(x.dt.day<=15))]).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 월 말 방문 비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('월말_방문비율', lambda x: np.mean(x.dt.day>15))]).reset_index()
features_prop.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('월말_방문비율', lambda x: np.mean(x.dt.day>15))]).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 월 별 구매 비율
f = pd.crosstab(train.CLNT_ID,train.date.dt.month,margins=True)
f = f.div(f.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f.columns,[str(x)+'월_방문비율' for x in f.columns]))).reset_index()
features_prop.append(f); display(f)

f_te = pd.crosstab(test.CLNT_ID,test.date.dt.month,margins=True)
f_te = f_te.div(f_te.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f_te.columns,[str(x)+'월_방문비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 계절별 구매 비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_prop.append(f); f

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te_prop.append(f_te); f_te

In [ ]:
# 반기 별 구매 비율
f = train.groupby('CLNT_ID')['date'].agg([
    ('전반기-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,3,4,5,6]))),
    ('후반기-구매비율', lambda x: np.mean( x.dt.month.isin([7,8,9,10,11,12])))
]).reset_index()
features_prop.append(f); f

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('전반기-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,3,4,5,6]))),
    ('후반기-구매비율', lambda x: np.mean( x.dt.month.isin([7,8,9,10,11,12])))
]).reset_index()
features_te_prop.append(f_te); f_te

In [ ]:
# 기기유형별 방문 횟수
f = pd.crosstab(train.CLNT_ID, train.DVC_CTG_NM,margins=True)
f = f.div(f.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f.columns,[x+'로_구매비율' for x in f.columns]))).reset_index()
features_prop.append(f);display(f)

f_te = pd.crosstab(test.CLNT_ID, test.DVC_CTG_NM,margins=True)
f_te = f_te.div(f_te.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f_te.columns,[x+'로_구매비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te);display(f_te)

In [ ]:
cust_train = train.CLNT_ID.unique()
cust_test = test.CLNT_ID.unique()
concat_data = pd.concat([train,test])

f1_list = list(train.CLAC1_NM.unique())
f2_list = list(train.CLAC2_NM.unique())
f3_list = list(train.CLAC3_NM.unique())
f1_list.append('All')
f2_list.append('All')
f3_list.append('All')

In [ ]:
f1 = pd.crosstab(concat_data.CLNT_ID, concat_data.CLAC1_NM, margins=True)[f1_list]
f1 = f1.div(f1.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f1.columns,[x+'_구매비율' for x in f1.columns]))).reset_index()
f2 = pd.crosstab(concat_data.CLNT_ID, concat_data.CLAC2_NM,margins=True)[f2_list]
f2 = f2.div(f2.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f2.columns,[x+'_구매비율' for x in f2.columns]))).reset_index()
#f3 = pd.crosstab(concat_data.CLNT_ID, concat_data.CLAC3_NM,margins=True)[f3_list]
#f3 = f3.div(f3.All,axis=0).iloc[:-1,:-1].rename(columns=dict(zip(f3.columns,[x+'_구매비율' for x in f3.columns]))).reset_index()

In [ ]:
# 상품 대분류별 구매 비율
f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')

features_prop.append(f);display(f)
features_te_prop.append(f_te);display(f_te)

In [ ]:
# 상품 중분류별 구매 비율
f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')

features_prop.append(f);display(f)
features_te_prop.append(f_te);display(f_te)

In [ ]:
# 상품 소분류별 구매 비율
#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')

#features_prop.append(f);display(f)
#features_te_prop.append(f_te);display(f_te)

In [ ]:
# 지역 대분류별 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[x+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='ZON_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te);display(f_te)

In [ ]:
# 지역 중분류별 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[x+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='CITY_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te);display(f_te)

In [ ]:
# 요일 별 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.day_name(), values='AMOUNT', aggfunc='sum', fill_value=0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.day_name(), values='AMOUNT', aggfunc='sum', fill_value=0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 월초/월말 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train['date'].dt.day>15, values='AMOUNT', aggfunc='sum', fill_value=0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns={False:'월초_구매금액비율',True:'월말_구매금액비율'}).reset_index()
features_prop.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test['date'].dt.day>15, values='AMOUNT', aggfunc='sum', fill_value=0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns={False:'월초_구매금액비율',True:'월말_구매금액비율'}).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 월 별 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns=train.date.dt.month, values='AMOUNT', aggfunc='sum', fill_value=0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=test.date.dt.month, values='AMOUNT', aggfunc='sum', fill_value=0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 계절 별 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([3,4,5]),'봄',
        np.where(train.date.dt.month.isin([6,7,8]),'여름',
        np.where(train.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='sum', fill_value=0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([3,4,5]),'봄',
        np.where(test.date.dt.month.isin([6,7,8]),'여름',
        np.where(test.date.dt.month.isin([9,10,11]),'가을','겨울'
        ))), values='AMOUNT', aggfunc='sum', fill_value=0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 전반기/후반기 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns=np.where(train.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                  '후반기'), values='AMOUNT', aggfunc='sum', fill_value=0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[str(x)+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f); display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns=np.where(test.date.dt.month.isin([1,2,3,4,5,6]),'전반기',
                                                                    '후반기'), values='AMOUNT', aggfunc='sum', fill_value=0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[str(x)+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te); display(f_te)

In [ ]:
# 사용기기별 구매금액비율
f = pd.pivot_table(data = train, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data.set_index('CLNT_ID')['총구매액'],axis=0)
f = f.rename(columns=dict(zip(f.columns,[x+'_구매금액비율' for x in f.columns]))).reset_index()
features_prop.append(f);display(f)

f_te = pd.pivot_table(data = test, index='CLNT_ID', columns='DVC_CTG_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data_te.set_index('CLNT_ID')['총구매액'],axis=0)
f_te = f_te.rename(columns=dict(zip(f_te.columns,[x+'_구매금액비율' for x in f_te.columns]))).reset_index()
features_te_prop.append(f_te);display(f_te)

In [ ]:
# 상품 대,중,소분류별 구매금액비율
data_all = pd.concat([data,data_te],axis=0).sort_values('CLNT_ID').reset_index(drop=True)

f1 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC1_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data_all.set_index('CLNT_ID')['총구매액'],axis=0)[f1_list[:-1]]
f1 = f1.rename(columns=dict(zip(f1.columns,[x+'_구매금액비율' for x in f1.columns]))).reset_index()
f2 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC2_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data_all.set_index('CLNT_ID')['총구매액'],axis=0)[f2_list[:-1]]
f2 = f2.rename(columns=dict(zip(f2.columns,[x+'_구매금액비율' for x in f2.columns]))).reset_index()
#f3 = pd.pivot_table(data = concat_data, index='CLNT_ID', columns='CLAC3_NM',values='AMOUNT', aggfunc='sum').fillna(0).div(data_all.set_index('CLNT_ID')['총구매액'],axis=0)[f3_list[:-1]]
#f3 = f3.rename(columns=dict(zip(f3.columns,[x+'_구매금액비율' for x in f3.columns]))).reset_index()

f = f1.query('CLNT_ID in @cust_train')
f_te = f1.query('CLNT_ID in @cust_test')
features_prop.append(f);display(f)
features_te_prop.append(f_te);display(f_te)

f = f2.query('CLNT_ID in @cust_train')
f_te = f2.query('CLNT_ID in @cust_test')
features_prop.append(f);display(f)
features_te_prop.append(f_te);display(f_te)

#f = f3.query('CLNT_ID in @cust_train')
#f_te = f3.query('CLNT_ID in @cust_test')
#features_prop.append(f);display(f)
#features_te_prop.append(f_te);display(f_te)

In [ ]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max),
                                            ('최소구매액', np.min)]) #1. 최소구매액

f['총구매액 대비 평균구매액'] = f['평균구매액']/f['총구매액'] # 2. 총구매액 대비 평균구매액
f['총구매액 대비 최대구매액'] = f['최대구매액']/f['총구매액'] # 3. 총구매액 대비 최대구매액
f['총구매액 대비 최소구매액'] = f['최소구매액']/f['총구매액'] # 4. 총구매액 대비 최소구매액

f = f.reset_index()
f = f.fillna(0)
f = f[['CLNT_ID','총구매액 대비 평균구매액','총구매액 대비 최대구매액','총구매액 대비 최소구매액']]
features_prop.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max),
                                            ('최소구매액', np.min)]) #1. 최소구매액

f_te['총구매액 대비 평균구매액'] = f_te['평균구매액']/f_te['총구매액'] # 2. 총구매액 대비 평균구매액
f_te['총구매액 대비 최대구매액'] = f_te['최대구매액']/f_te['총구매액'] # 3. 총구매액 대비 최대구매액
f_te['총구매액 대비 최소구매액'] = f_te['최소구매액']/f_te['총구매액'] # 4. 총구매액 대비 최소구매액

f_te = f_te.reset_index()
f_te = f_te.fillna(0)
f_te = f_te[['CLNT_ID','총구매액 대비 평균구매액','총구매액 대비 최대구매액','총구매액 대비 최소구매액']]
features_te_prop.append(f_te) ; display(f_te)

In [ ]:
f = train.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('총접속시간', np.sum),
                                                   ('평균접속시간', lambda x : np.round(np.mean(x))),
                                                   ('최대접속시간', np.max),
                                                   ('최소접속시간', np.min)])

f['총접속시간 대비 평균접속시간'] = f['평균접속시간']/f['총접속시간']
f['총접속시간 대비 최대접속시간'] = f['최대접속시간']/f['총접속시간']
f['총접속시간 대비 최소접속시간'] = f['최소접속시간']/f['총접속시간']

f = f.reset_index()
f = f.fillna(0)
f = f[['CLNT_ID','총접속시간 대비 평균접속시간','총접속시간 대비 최대접속시간','총접속시간 대비 최소접속시간']]
features_prop.append(f);display(f)

f_te = test.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('총접속시간', np.sum),
                                                     ('평균접속시간', lambda x : np.round(np.mean(x))),
                                                     ('최대접속시간', np.max),
                                                     ('최소접속시간', np.min)])

f_te['총접속시간 대비 평균접속시간'] = f_te['평균접속시간']/f_te['총접속시간']
f_te['총접속시간 대비 최대접속시간'] = f_te['최대접속시간']/f_te['총접속시간']
f_te['총접속시간 대비 최소접속시간'] = f_te['최소접속시간']/f_te['총접속시간']

f_te = f_te.reset_index()
f_te = f_te.fillna(0)
f_te = f_te[['CLNT_ID','총접속시간 대비 평균접속시간','총접속시간 대비 최대접속시간','총접속시간 대비 최소접속시간']]
features_te_prop.append(f_te);display(f_te)

In [ ]:
data_prop = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})

for f in tqdm(features_prop) :
    data_prop = pd.merge(data_prop, f, how='left')

In [ ]:
data_te_prop = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})

for f_te in tqdm(features_te_prop) :
    data_te_prop = pd.merge(data_te_prop, f_te, how='left')

In [ ]:
display(data_prop)
display(data_te_prop)

In [ ]:
data_prop.to_csv('/content/drive/MyDrive/D&A_ML_Competition/data_prop.csv')
data_te_prop.to_csv('/content/drive/MyDrive/D&A_ML_Competition/data_te_prop.csv')

# <font color='CC3D3D'>Final Features</font>

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/D&A_ML_Competition/data1.csv')
data_te = pd.read_csv('/content/drive/MyDrive/D&A_ML_Competition/data_te1.csv')

In [ ]:
data_final = pd.merge(data,data_prop,on='CLNT_ID')
data_final.to_csv('/content/drive/MyDrive/D&A_ML_Competition/data_final.csv')

In [ ]:
data_te_final = pd.merge(data_te,data_te_prop,on='CLNT_ID')
data_te_final.to_csv('/content/drive/MyDrive/D&A_ML_Competition/data_te_final.csv')